This notebook looks at the impact of flux-normalising H, and also verifies that the deconvolution is stable if I fix certain planes in the reconstruction, subtract their effect from the input image, and then repeat the deconvolution for the remaining planes. That is something I intend to rely on to speed up my PIV analysis of Nils' huge dataset.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing
import tifffile
import sys, time, os, csv
import cProfile, pstats
from jutils import tqdm_alias as tqdm

import psfmatrix, lfimage
import projector, lfdeconv
import special_fftconvolve as special
import jutils as util
import py_light_field as plf

In [ ]:
projectorClass = projector.Projector_allC
maxiter = 8
inputImage = lfimage.LoadLightFieldTiff('/Users/jonny/Movies/Nils files/Rectified/Left/Cam_Left_40_X1_N19.tif')
hMatrix = psfmatrix.LoadMatrix('PSFmatrix/reducedPSFMatrix_M22.2NA0.5MLPitch125fml3125from-156to156zspacing4Nnum19lambda520n1.33.mat')

### Stability investigations
Now that I have flux-normalised H, does the deconvolution converge to a stable image (importantly: with a stable absolute amplitude) as we run increasing numbers of iterations? The maximum values do seem to rise a little, but the sum does at least seem to remain approximately the same. **Conclusion**: I am reassured that the sum and maximum remain almost unchanged.

Can we do an initial reconstruction, subtract the contribution from most of the planes to the camera image, and then perform subsequent reconstructions on just a few planes of interest in the dataset? (This is what I want to do with my flow analysis, where I don't want to be having to reconstruct the whole volume during my iterative optimization, when I am only interested in the artefact-corrupted planes). **Conclusion**: yes, that does seem to work - I get almost exactly the same reconstruction as in the original, full reconstruction.

In [ ]:
hMatrixSmall = psfmatrix.LoadMatrix('PSFmatrix/fdnormPSFMatrix_M22.2NA0.5MLPitch125fml3125from-12to0zspacing4Nnum19lambda520n1.33.mat')
hMatrixSmall1 = psfmatrix.LoadMatrix('PSFmatrix/fdnormPSFMatrix_M22.2NA0.5MLPitch125fml3125from-12to-8zspacing4Nnum19lambda520n1.33.mat')
hMatrixSmall2 = psfmatrix.LoadMatrix('PSFmatrix/fdnormPSFMatrix_M22.2NA0.5MLPitch125fml3125from-4to0zspacing4Nnum19lambda520n1.33.mat')

In [ ]:
# Generate a synthetic four-plane image
obj = tifffile.imread('Data/03_Reconstructed/exampleData/definitive_worm_crop_X15_backproject.tif')
obj = obj[7:11,np.newaxis].astype(np.float32)
im = lfdeconv.ForwardProjectACC(hMatrixSmall, obj, progress=tqdm, projector=projectorClass())

In [ ]:
# Deconvolve the full four planes
Htf = lfdeconv.BackwardProjectACC(hMatrixSmall, im, progress=tqdm, projector=projectorClass())
obj8 = lfdeconv.DeconvRL(hMatrixSmall, Htf, 8, Htf.copy(), logPrint=False)

In [ ]:
# Sanity check that intensities remain approx the same as we iterate the deconvolution.
if True:
    obj10 = lfdeconv.DeconvRL(hMatrixSmall, Htf, 2, obj8, logPrint=False)
    obj20 = lfdeconv.DeconvRL(hMatrixSmall, Htf, 10, obj10, logPrint=False)
    
    print(np.max(obj8), np.sum(obj8))
    print(np.max(obj10), np.sum(obj10))
    print(np.max(obj20), np.sum(obj20))

    plt.figure()
    plt.imshow(obj8[0,0])
    plt.show()
    plt.figure()
    plt.imshow(obj10[0,0])
    plt.show()    
    plt.figure()
    plt.imshow(obj20[0,0])
    plt.show()        

In [ ]:
# Forward-project just the first, and second, halves of that reconstructed object
imFirstHalf = lfdeconv.ForwardProjectACC(hMatrixSmall1, obj8[0:2], progress=tqdm, projector=projectorClass())
imSecondHalf = lfdeconv.ForwardProjectACC(hMatrixSmall2, obj8[2:4], progress=tqdm, projector=projectorClass())

In [ ]:
# Subtract the forward-projection of the first two planes from our original camera image,
# and try reconstructing the second two planes in isolation.
imPartial = im - imFirstHalf
obj8SecondHalf = lfdeconv.DeconvRL(hMatrixSmall2, None, 8, None, im=imPartial, logPrint=False)

In [ ]:
# Compare the reconstructions
plt.imshow(obj8[2,0]); plt.colorbar(); plt.show()
plt.imshow(obj8SecondHalf[0,0]); plt.colorbar(); plt.show()
plt.imshow(obj8[2,0] - obj8SecondHalf[0,0]); plt.colorbar(); plt.show()

plt.imshow(obj8[3,0]); plt.colorbar(); plt.show()
plt.imshow(obj8SecondHalf[1,0]); plt.colorbar(); plt.show()
plt.imshow(obj8[3,0] - obj8SecondHalf[1,0]); plt.colorbar(); plt.show()

### Compare the deconvolutions as obtained from various different PSFs
It's clear that the max bug does increase the artefacts in the native focal plane, although it's not catastrophic. It's hard to quantify how much difference the normalisation makes, but it is not a huge visual impact, and the relative differences seem to be on the 10% level.

In [ ]:
theirs = tifffile.imread('/Users/jonny/Movies/Nils files/Single-view-reconstructions/Cam_Left_40_uncompressed.tif')
mineNorm = tifffile.imread('Data/03_Reconstructed/Left/Cam_Left_40_X1_N19.tif')
mineUnnorm = tifffile.imread('Data/03_Reconstructed/Left/Cam_Left_40_X1_N19_unnorm.tif')
mineFDnorm = tifffile.imread('Data/03_Reconstructed/Left/Cam_Left_40_X1_N19_fdnorm.tif')

In [ ]:
for cc in [20, 38, 39, 40, 55]:
    plt.figure(figsize=(10,10)); plt.title('theirs %d'%cc)
    plt.imshow(theirs[cc])
    plt.colorbar(); plt.show()
    if False:
        plt.figure(figsize=(10,10)); plt.title('mineNorm %d'%cc)
        plt.imshow(mineNorm[cc])
        plt.colorbar(); plt.show()
        plt.figure(figsize=(10,10)); plt.title('mineUnnorm %d'%cc)
        plt.imshow(mineUnnorm[cc])
        plt.colorbar(); plt.show()
    plt.figure(figsize=(10,10)); plt.title('mineFDnorm %d'%cc)
    plt.imshow(mineFDnorm[cc])
    plt.colorbar(); plt.show()
    frac = mineUnnorm[cc]/mineFDnorm[cc]
    plt.figure(figsize=(10,10)); plt.title('mine unnorm/fdnorm %d (%f)'%(cc, (np.max(frac)-np.min(frac))/np.min(frac)))
    plt.imshow(frac)
    plt.colorbar(); plt.show()
    frac = mineUnnorm[cc]/theirs[cc]
    plt.figure(figsize=(10,10)); plt.title('mine unnorm/theirs %d (%f)'%(cc, (np.max(frac)-np.min(frac))/np.min(frac)))
    plt.imshow(frac)
    plt.colorbar(); plt.show()    
    
plt.figure(figsize=(10,10)); plt.title('theirs MIP')
plt.imshow(np.max(theirs, axis=0))
plt.colorbar(); plt.show()
plt.figure(figsize=(10,10)); plt.title('mine unnorm MIP')
plt.imshow(np.max(mineUnnorm, axis=0))
plt.colorbar(); plt.show()
plt.figure(figsize=(10,10)); plt.title('mine FDnorm MIP')
plt.imshow(np.max(mineFDnorm, axis=0))
plt.colorbar(); plt.show()

frac = np.max(mineUnnorm,axis=0)/np.max(mineFDnorm,axis=0)
plt.figure(figsize=(10,10)); plt.title('MIP mine unnorm/fdnorm MIP %f)'%((np.max(frac)-np.min(frac))/np.min(frac)))
plt.imshow(frac)
plt.colorbar(); plt.show()
frac = np.max(mineUnnorm,axis=0)/np.max(theirs,axis=0)
plt.figure(figsize=(10,10)); plt.title('MIP mine unnorm/theirs MIP (%f)'%((np.max(frac)-np.min(frac))/np.min(frac)))
plt.imshow(frac)
plt.colorbar(); plt.show()    